In [242]:
import re
import numpy as np
import codecs
import math
import pandas as pd
from sklearn.model_selection import train_test_split
f = codecs.open("spam.csv", "r", "ISO-8859-1")
allwords = []

f.readline()

categ_size = 2;
for line in f:
    line_v = line.strip().lower().split(",")
    line = ' '.join(line_v[1:])
    words = re.split('[^a-z\d]',line)
    words.remove('')
    categ_index = 0
    if line_v[0] == 'ham':
        categ_index = 1;
    allwords.append([categ_index, words])
    

    

train_set, test_set = train_test_split(allwords, test_size = 0.2)

allwords_categ = [dict(), dict()]
documents_categ = [0, 0]
windex = dict()
wcount = dict()
i = 1;
for allword in train_set:
    categ_index = allword[0]
    words = allword[1]
    documents_categ[categ_index] += 1;
    for word in words:
        if word not in windex :
            windex[word] = i
            wcount[word] = 0
            i = i + 1
        wcount[word] += 1
        if word not in allwords_categ[categ_index] :
            allwords_categ[categ_index][word] = 0
        allwords_categ[categ_index][word] += 1



In [243]:
print(documents_categ)

categ_w = np.zeros((categ_size, len(windex)+1))

for word in windex:
    for categ_index in [0, 1]:
        if word in allwords_categ[categ_index]:
            categ_w[categ_index][windex[word]] = math.log(allwords_categ[categ_index][word]/wcount[word])
        else:
            categ_w[categ_index][windex[word]] = math.log(0.0000001)
            
        
eval_x = np.zeros((categ_size, categ_size))
for t_word in test_set:
    categ_index = t_word[0]
    words = t_word[1]
    test_x = np.zeros((1, len(windex)+1))
    for word in words:
        if word in windex:
            test_x[0, windex[word]] = 1
        else:
            test_x[0, 0] = 1
    
    #spam - 0
    spam_p = math.exp(np.dot(test_x, categ_w[0])[0])*(documents_categ[0]/(documents_categ[0]+documents_categ[1]))
    ham_p = math.exp(np.dot(test_x, categ_w[1])[0])*(documents_categ[1]/(documents_categ[0]+documents_categ[1]))
    
    if spam_p < ham_p:
        if categ_index == 1 :
            eval_x[1,1] += 1
        else:
            eval_x[1,0] += 1
    else:
        if categ_index == 0 :
            eval_x[0,0] += 1
        else:
            eval_x[0,1] += 1

print(eval_x)
print(eval_x[0,0]/(eval_x[0,1]+eval_x[0,0]))
print(eval_x[0,0]/(eval_x[1,0]+eval_x[0,0]))
    
    

        

[609, 3850]
[[ 116.    2.]
 [  24.  973.]]
0.983050847458
0.828571428571
